In [1]:
import tweepy 
import re,string
import csv
from sklearn import metrics
import csv
import pickle 
import joblib
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from statistics import mode

np.random.seed(500)

In [2]:
consumer_key = "7LpNzY0vxJLJ6G3urHqWf2aOV"
consumer_secret = "pVTsVgrpFDMWO23ieJeb4Y3Fezs9bLLssk4PeZds4K7VUsaIYR"
access_token = "1224024091943829504-yt4GWoJfJQz18ocgjfg9AfwQE8BaIW"
access_token_secret = "ZRRv9Ch6MVHG6MPKKuPoI4dVPBxD3rVyxaZC0tdYHqVwx"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = API(auth)

In [3]:
#class value 1 for negative
#class value 2 for pilot
#class value 3 for architect
#class value 4 for photographer
#class value 5 for finance

In [4]:
#random
neg=pd.read_csv('dataset140.csv',nrows=2000)
negtweets=neg['Text'].tolist()
values=[1]*len(negtweets)
neg_text=[]
for i in negtweets:
    i=re.sub(r"https:(\/\/t\.co\/([A-Za-z0-9]|[A-Za-z]){10})", "", i)
    i=re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", "",i)
    neg_text.append(i)
random=pd.DataFrame(neg_text,columns=['Text'])
random['Class']=values

In [5]:
def getTweets(filename,classval):
    df=pd.read_csv(filename)
    names=df['Name'].values.tolist()
    alltweets=[]
    values=[]
    for i in names:
        try:
            tweets=api.user_timeline(screen_name=i,count=400)#,tweet_mode="extended"
        except tweepy.TweepError:
            print("invalid")
        for tweet in tweets:
            if tweet.lang == "en":
                alltweets.append(tweet.text)
            
    values=[classval]*len(alltweets)
    text=[]
    for i in alltweets:
        i=re.sub(r"https:(\/\/t\.co\/([A-Za-z0-9]|[A-Za-z]){10})", "", i)
        i=re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", "",i)
        text.append(i)
    pos=pd.DataFrame(text,columns=['Text'])
    pos['Class']=values
    return pos

In [6]:
pilot=getTweets('pilotcsv.csv',2)
architect=getTweets('architect.csv',3)
photographer=getTweets('photographer.csv',4)
finance=getTweets('finance.csv',5)

invalid


In [7]:
d=[random,architect,pilot,photographer,finance]
result = pd.concat(d)
result.to_csv('result.csv',index=False)
result=pd.read_csv(r"result.csv",encoding='latin-1')
result

,Text,Class
0,is upset that he can't update his Facebook by ...,1
1,I dived many times for the ballManaged to sav...,1
2,my whole body feels itchy and like its on fire,1
3,noit's not behaving at alli'm madwhy am i her...,1
4,not the whole crew,1
...,...,...
7393,Time passes and confirmation bias has differe...,5
7394,Current updated global and U.SGDP forecasts fr...,5
7395,In smaller 19th century redwood mills one stea...,5
7396,Answer to Fridayâs artifact quizItâs the â...,5


In [8]:
result['Text'].dropna(inplace=True)# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
result['Text'] = [entry.lower() for entry in result['Text']]# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
result['Text']= [word_tokenize(entry) for entry in result['Text']]# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [9]:
for index,entry in enumerate(result['Text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    result.loc[index,'text_final'] = str(Final_words)

In [10]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(result['text_final'],result['Class'],test_size=0.2)

In [11]:
# Encoder = LabelEncoder()
# Train_Y = Encoder.fit_transform(Train_Y)
# Test_Y = Encoder.fit_transform(Test_Y)
# print(len(Test_X))

In [12]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(result['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)
print(Tfidf_vect.vocabulary_)
print(Test_X_Tfidf)

{'upset': 4565, 'ca': 646, 'update': 4557, 'facebook': 1524, 'might': 2631, 'cry': 1045, 'result': 3612, 'school': 3818, 'today': 4395, 'dive': 1234, 'many': 2539, 'time': 4383, 'save': 3800, 'rest': 3607, 'go': 1816, 'bound': 564, 'whole': 4824, 'body': 533, 'feel': 1578, 'itchy': 2224, 'like': 2430, 'fire': 1614, 'noit': 2822, 'behave': 467, 'alli': 128, 'see': 3848, 'crew': 1027, 'need': 2776, 'hug': 2052, 'hey': 1977, 'long': 2473, 'bit': 510, 'lol': 2469, 'fine': 1610, 'thanks': 4331, 'nope': 2832, 'que': 3381, 'spring': 4095, 'break': 578, 'plain': 3101, 'snowing': 4010, 'ear': 1319, 'could': 982, 'bear': 456, 'watch': 4711, 'think': 4349, 'loss': 2481, 'never': 2791, 'talk': 4273, 'anymore': 256, 'would': 4922, 'gun': 1892, 'really': 3491, 'wish': 4877, 'get': 1794, 'miss': 2656, 'premiere': 3181, 'scene': 3813, 'hurt': 2062, 'severely': 3884, 'film': 1599, 'director': 1211, 'cut': 1066, 'file': 1597, 'tax': 4286, 'ahh': 95, 'ive': 2230, 'always': 147, 'want': 4693, 'love': 2486

In [13]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  74.72972972972973


In [14]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print(len(predictions_SVM))
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

1480
SVM Accuracy Score ->  71.89189189189189


In [15]:

def getCareerByText(test):
    # Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
    test = test.lower()# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
    test = word_tokenize(test)# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(test):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            words.append(word_Final)
#     words

    words_fit=Tfidf_vect.transform(words)
#         print(words_fit)
    answer=SVM.predict(words_fit)
    print(answer)
    n=mode(answer)
    if n==1:
        return "random"
    elif n==2:
        return "pilot"
    elif n==3:
        return "architect"
    elif n==4:
        return "photographer"
    elif n==5:
        return "finance"

In [36]:
ans=getCareerByText("nice pictures and beautiful shot and a well designed photo shoot")
print(ans)

[2 4 2 4 2 3 4 4]
photographer


In [17]:
def getCareerById(username):
    alltweets=[]
    try:
        tweets=api.user_timeline(screen_name=username,count=200)#,tweet_mode="extended"
    except tweepy.TweepError:
        print("invalid")
    for tweet in tweets:
        if tweet.lang == "en":
            alltweets.append(tweet.text)
            
    text=[]
    for i in alltweets:
        i=re.sub(r"https:(\/\/t\.co\/([A-Za-z0-9]|[A-Za-z]){10})", "", i)
        i=re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", "",i)
        text.append(i)
    result=pd.DataFrame(text,columns=['Text'])
    result.to_csv('user.csv',index=False)
    result=pd.read_csv(r"user.csv",encoding='latin-1')
    result['Text'].dropna(inplace=True)# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
    result['Text'] = [entry.lower() for entry in result['Text']]# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
    result['Text']= [word_tokenize(entry) for entry in result['Text']]# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    for index,entry in enumerate(result['Text']):
    # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
    # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
            if word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
        result.loc[index,'text_final'] = str(Final_words)
#     print(result['text_final'])
    res_fit=Tfidf_vect.transform(result['text_final'])
#     print(res_fit)
    answer=SVM.predict(res_fit)
    print(answer)
    n=mode(answer)
    if n==1:
        return "random"
    elif n==2:
        return "pilot"
    elif n==3:
        return "architect"
    elif n==4:
        return "photographer"
    elif n==5:
        return "finance"

In [18]:
ans=getCareerById('shesgotwings777')
print(ans)

[2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 5 2 2 2 2 2 2 2 2 2 2 2 1 2 2
 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 2 2
 2 2 2 2 2 2 1 2 4 2 2 2 2 2 1 2 2 2 4 2 1 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2
 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 1 2 2 2 2 2 2 2 2 1 2 2 2 2 2 4 2 1 2 1 2 1 2 2 2 2 5 1 2 2 2 2 2 2
 1 2 2 2 2]
pilot


In [33]:
# Save the model as a pickle in a file 
joblib.dump(SVM, 'svmcareer.pkl') 
joblib.dump(Naive, 'nbcareer.pkl') 
joblib.dump(Tfidf_vect,'vocab.pkl')

['vocab.pkl']